In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
with open('art_links') as f:
    data = f.readlines()
    data = [data.rstrip('\n') for data in open('art_links')]

data = filter(None, data)
data = data[12000:]

In [ ]:
soup_objs = []

for line in data:
    response = requests.get(line)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
from collections import defaultdict

client = MongoClient()
db = client.paintings

artwork_test = db.artwork_test


for soup in soup_objs: 
    art_dict = defaultdict(list)
    dets = soup.findAll('dt', class_='collection-details__tombstone--label')
    for s in dets:
        art_dict[s.text].append(s.nextSibling.nextSibling.text)
    art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
    #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
    art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
    art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
    images = soup.findAll('img')
    for i in images:
        if 'ng-src' in str(i):
            art_dict['image']=i.encode('utf-8')
    artwork_test.insert_one(art_dict)

In [ ]:
artwork_test.count()

In [ ]:
type(artwork_test)

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork_test.find()))

In [ ]:
import numpy as np

df['description'] = df['description'].str.replace('<div class="collection-details__label">\n</div>', '')
df['description'] = df['description'].str.replace('<div class="collection-details__label">\r\n', '')
df['description'] = df['description'].str.replace('<br/><br/>', '')
df['description'] = df['description'].str.replace('\r\n    </div>', '')
df['gallery'] = df['gallery'].str.replace('<div class="collection-details__location">\n<svg class="icon">\n<use xlink:href="/dist/icons.svg#ico-marker" xmlns:xlink="http://www.w3.org/1999/xlink"></use>\n</svg>\r\n', '')
df['gallery'] = df['gallery'].str.replace('\r\n    </div>', '')
df['title'] = df['title'].str.replace('<h1 class="collection-details__object-title">', '')
df['title'] = df['title'].str.replace('</h1>', '')
df['title'] = df['title'].str.replace('\r\n<br/>', '')
#df['gallery'] = df['gallery'].str.replace('        On view at The Met Fifth Avenue in Gallery ', '')

In [ ]:
len(df)

In [ ]:
df.head(100)

In [ ]:
df.to_csv('my_df_1.csv', sep='\t', encoding='utf-8')

In [ ]:
## NLTK With NLP

In [ ]:
About SKLearn Text Processing:
- array: each string is an element
- creating a feature for every word
- vectorizer: transforms text into feature names and gives counts of each word in a vector
- raw text to feature count

In [ ]:
import nltk
nltk.download('maxent_treebank_pos_tagger')

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('words')

In [ ]:
### Sentence Tokenization

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = [sent_tokenize(x.lower()) for x in df['description']]
print sentences[0]

In [ ]:
### Word Tokenziation

In [ ]:
from nltk.tokenize import word_tokenize

words = [word_tokenize(x) for x in df['description']]
print words[0]

In [ ]:
### Parts of Speech Tagging

In [ ]:
nltk.download()

In [ ]:
from nltk.tag import pos_tag
word_tags = word_tokenize(df['description'].str.lower()[0])
pos_tag(word_tags)

In [ ]:
## TextBlob

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob

In [ ]:
blob = [TextBlob(x) for x in (df['description'])]
blob[10]

In [ ]:
### Sentiment w/ TextBlob


In [ ]:
blob[10].sentiment

In [ ]:
### Stemming w/ TextBlob

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()

for word in blob[0].words:
    print stemmer.stem(word)

In [ ]:
## Count Vectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
About Count Vectorizer:
- term frequency
- converts words to a matrix of token counts
- sparse matrix

In [ ]:
df_lower = [i.lower() for i in df['description']]

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
count_vect = vectorizer.fit_transform([i for i in df['description']])
#count_vect = vectorizer.fit_transform(df_lower)
count_vect

In [ ]:
count_vect[:100].toarray()

In [ ]:
## Bigrams

In [ ]:
Bigrams: break down text into two-word phrases

In [ ]:
bigram_vectorizer = CountVectorizer(stop_words= 'english', ngram_range=(2,2),min_df=1)
bigram_vectorizer.fit_transform(df_lower[:100]).toarray()

In [ ]:
bigram_vectorizer.get_feature_names()[200:210]

In [ ]:
## TFIDF

In [ ]:
Term Frequency, Inverse Document Frequency:
- a word is more important the more times it appears in a single document (high score)
- BUT if a word appears in many documents, not a unique identifier (low score)
- number of times a word appears in a single document / (divided by) number of documents which the word occurs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit_transform([i for i in df_lower[:100]]).toarray()

In [ ]:
## Distance

In [ ]:
Euclidean:
- length of the path connecting 2 points (given by pythagorean theorum)
- best for when data is dense or continuous  
  
Manhattan:
- distance between 2 points is the sum of the absolute differences of their Cartesian coordinates(sum of the differences between x and y coordinates)
- measured along right angles (like city blocks)  
  
Cosine Similarity:
- finding cosine angle between 2 objects (normalized dot product)
- good for sparse vectors 
- the closer to 1, the more alike
  
Because my matrix is sparse, I'll use cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

#text = ['hi mike', 'hi hi hi mike mike mike']
#X = vectorizer.fit_transform(text)

#print cosine_distances(X[0], X[1])

#print cosine_distances()
print cosine_distances(count_vect[2000], count_vect[80])

In [ ]:
count_vect[50]

In [ ]:
## Recommender System

In [ ]:
Item-Item Collaborative Filtering:  
- find the most similar items, then combine that in formation w/ the user's ratings for items to generate recommendations
- will use an implicit ratings matrix (did the user like it or not?)
- will not use explicit matrix: user ratings
- matrix: each row is a different user, each column is a painting w/ 0 or 1
- create a matrix w/ each painting as similarity score

In [ ]:
## OR SVD